In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt

from copy import deepcopy
import pandas as pd

from sklearn.model_selection import ParameterGrid

import seaborn as sns

import matplotlib as mpl
os.environ['PATH'] = os.environ['PATH'] + ':/Library/TeX/texbin'
mpl.rcParams.update(mpl.rcParamsDefault)


In [ ]:
param_grid_values = {
    'look_distance' : [0.1],
    'kernel_bandwidthLon' : [ 0.1, 0.15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7, .75],
    'kernel_bandwidthLat' : [ 0.02, 0.04 , 0.06, .08, .1, .12],
    'n_iterations' : [20],
}

param_grid = ParameterGrid(param_grid_values)
num_cpu = 20
input_type = "confidence_map"

In [ ]:
param_optim_folder2 = '../datasets/classification/param_optimP2'

grid_image_out_dict = {}
grid_image_out_dict_stats = {}

my_len = 0

for param_idx, params in enumerate(param_grid):
    print(param_idx, params)

    out_file1 = os.path.join(param_optim_folder2, f'image_outdict_dist{params["look_distance"]}_Lon{params["kernel_bandwidthLon"]}_lat{params["kernel_bandwidthLat"]}_iter{params["n_iterations"]}.json')
    out_file2 = os.path.join(param_optim_folder2, f'image_outdictStats_dist{params["look_distance"]}_Lon{params["kernel_bandwidthLon"]}_lat{params["kernel_bandwidthLat"]}_iter{params["n_iterations"]}.json')
    
    with open(out_file1,'r') as f:
        image_out_dict = json.load(f)
    with open(out_file2,'r') as f:
        image_out_dict_stats = json.load(f)

    my_len = len(list(image_out_dict.keys()))

    grid_image_out_dict[param_idx] = deepcopy(image_out_dict)
    grid_image_out_dict_stats[param_idx] = deepcopy(image_out_dict_stats)
    
    

In [ ]:
import collections
from collections import Counter

cur_out_stats = {
            # General info
            'num_DB_groups':0,
            'num_MS_groups':0,

            'matches':0,

            # MS with DB matching info
            'unmatched_db':0,
            'multiDB_singleMS': 0,
            'db_too_far':0,


            'unmatched_ms':0,
            'bad_ms':0,
            'ms_too_far':0,

            "ious":0,
            "distances":0

    }

stats_keys = cur_out_stats.keys()

should_check =  list(stats_keys) + [
                                        'num_rejects_all',
                                        'num_optimizable_rejects',
                                        'rate_optimizable_rejects',
                                    ]

gridsearch_dict_per_img = { k : np.zeros((len(param_grid_values['kernel_bandwidthLon']), 
                                   len(param_grid_values['kernel_bandwidthLat']),
                                      my_len
                                   ))
                    for k in should_check}

should_check_global = list(stats_keys) + [
                                            'num_rejects_all',
                                            'num_optimizable_rejects',
                                            'rate_optimizable_rejects',
                                         ]
gridsearch_dict_total = { k : np.zeros((len(param_grid_values['kernel_bandwidthLon']), 
                                   len(param_grid_values['kernel_bandwidthLat'])
                                   ))
                    for k in should_check_global}

grid_search_reject_distribs = {}


for param_idx, params in enumerate(param_grid):
#     param_idx= str(param_idx)
    image_out_dict = grid_image_out_dict[param_idx]
    image_out_dict_stats = grid_image_out_dict_stats[param_idx]
    
    # General info
    num_DB_groups_per_image = np.array([item['num_DB_groups'] for k,item in image_out_dict_stats.items()])
    num_MS_groups_per_image = np.array([item['num_MS_groups'] for k,item in image_out_dict_stats.items()])

    diff_num_groups = num_DB_groups_per_image - num_MS_groups_per_image # should be shown on histogram , closer to 0 is better


    # matches
    num_MSmatchesDB = np.array([ np.sum(np.array(v['matches'])>-1) for k,v in image_out_dict_stats.items()])
    
    num_unmatched_db = np.array([len(v['unmatched_db']) for k,v in image_out_dict_stats.items()])
    num_multiDB_singleMS = np.array([len(v['multiDB_singleMS']) for k,v in image_out_dict_stats.items()])
    num_db_too_far = np.array([len(v['db_too_far']) for k,v in image_out_dict_stats.items()])


    num_unmatchedMS = np.array([len(v['unmatched_ms']) for k,v in image_out_dict_stats.items()])
    num_badMS = np.array([len(v['bad_ms']) for k,v in image_out_dict_stats.items()])
    num_ms_too_far = np.array([len(v['ms_too_far']) for k,v in image_out_dict_stats.items()])
    
  
    num_optimizable_rejects = num_badMS + num_multiDB_singleMS 
    num_rejects_all = num_optimizable_rejects + num_db_too_far

    # get indices of current params
    kernel_bandwidthLon_idx = param_grid_values['kernel_bandwidthLon'].index(params['kernel_bandwidthLon'])
    kernel_bandwidthLat_idx = param_grid_values['kernel_bandwidthLat'].index(params['kernel_bandwidthLat'])
    ############################
    # Fill per-image dictinnary 
    ############################

    # number of DB groups
    gridsearch_dict_per_img['num_DB_groups'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_DB_groups_per_image
    # number of MS groups
    gridsearch_dict_per_img['num_MS_groups'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_MS_groups_per_image

    # number of matches
    # print(num_MSmatchesDB.shape)
    # print(num_MSmatchesDB)
    gridsearch_dict_per_img['matches'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_MSmatchesDB
    # number of unmatched DB groups
    gridsearch_dict_per_img['unmatched_db'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_unmatched_db
    # number of multi DB groups matched to single MS group
    gridsearch_dict_per_img['multiDB_singleMS'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_multiDB_singleMS
    # number of DB groups too far from MS group
    gridsearch_dict_per_img['db_too_far'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_db_too_far
    # number of unmatched MS groups
    gridsearch_dict_per_img['unmatched_ms'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_unmatchedMS
    # number of bad MS groups
    gridsearch_dict_per_img['bad_ms'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_badMS
    # number of MS groups too far from DB group
    gridsearch_dict_per_img['ms_too_far'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_ms_too_far


    gridsearch_dict_per_img['num_optimizable_rejects'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_optimizable_rejects
    gridsearch_dict_per_img['num_rejects_all'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = num_rejects_all
     
    ############################
    # Fill total dictionnary
    ############################
    # number of DB groups
    gridsearch_dict_total['num_DB_groups'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_DB_groups_per_image)
    # number of MS groups
    gridsearch_dict_total['num_MS_groups'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_MS_groups_per_image)

    # number of matches
    gridsearch_dict_total['matches'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_MSmatchesDB)
    # number of unmatched DB groups
    gridsearch_dict_total['unmatched_db'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_unmatched_db)
    # number of multi DB groups matched to single MS group
    gridsearch_dict_total['multiDB_singleMS'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_multiDB_singleMS)
    # number of DB groups too far from MS group
    gridsearch_dict_total['db_too_far'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_db_too_far)
    # number of unmatched MS groups
    gridsearch_dict_total['unmatched_ms'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_unmatchedMS)
    # number of bad MS groups
    gridsearch_dict_total['bad_ms'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_badMS)
    # number of MS groups too far from DB group
    gridsearch_dict_total['ms_too_far'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_ms_too_far)
    
    

    gridsearch_dict_total['num_optimizable_rejects'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_optimizable_rejects)
    gridsearch_dict_total['num_rejects_all'][kernel_bandwidthLon_idx,kernel_bandwidthLat_idx] = np.sum(num_rejects_all)

#     break
    
